The ``pycentral`` subfolder contains the Aruba Central Python modules. Each module contains multiple Python classes. Each Class is a representation of some of the Aruba Central's API categories as mentioned in the [API Reference section](https://developer.arubanetworks.com/aruba-central/reference). Each class has its own function definitions that are used to make a single REST API call. 

In addition to using the modules provided by the package, there is also a module `base.py` containing function `ArubaCentralBase.command()`. This function can make an API call to any available REST API using the endpoint URL, HTTP Method, HTTP query params and HTTP payload as required by an API endpoint. The REST API calls are made using the Python *requests* library by `ArubaCentralBase.command()`, as supported by Aruba Central API Gateway endpoints. 

## Gathering Data

Gathering variables required for the package base class `ArubaCentralBase`.

- **base_url**

    Go to `Account Home -> API Gateway -> APIs`. *All Published APIs Table* will show URL under *DOCUMENTATION* column. Truncate this to end with '.com'. For example, `https://apigw-prod2.central.arubanetworks.com/swagger/central/` must be truncated to `https://apigw-prod2.central.arubanetworks.com`

- **access_token**

    We already saw in the first part how to obtain the `access_token` through the API Gateway.
    
 
### Task 1
Sample pycentral script central_api.py

In [3]:
# We will use the base module from pycentral package in this script
from pycentral.base import ArubaCentralBase

# Create an instance of ArubaCentralBase using API Gateway's Base URL and the API access token

central_info = {
    "base_url": "<api-gateway-base-url>",
    "token": {
        "access_token": "<api-gateway-access-token>"
    }
}
ssl_verify = True
central = ArubaCentralBase(central_info=central_info,
                           ssl_verify=ssl_verify)

# Sample API call using 'ArubaCentralBase.command()'
'''
ArubaCentralBase module is used to run any available API on Central.
There are other modules available which are API specific modules.
For more information regarding various available modules and its classes, visit:
https://pycentral.readthedocs.io/en/latest/pycentral.html#module-pycentral.base
'''

# GET groups from Aruba Central
'''apiPath, apiMethod and apiParams are obtained from the
Central API documentation page aka Swagger, discussed in Lab1
'''
apiPath = "/configuration/v2/groups"
apiMethod = "GET"
apiParams = {
    "limit": 20,
    "offset": 0
}
base_resp = central.command(apiMethod=apiMethod, 
                            apiPath=apiPath,
                            apiParams=apiParams)
print(base_resp)

ModuleNotFoundError: No module named 'pycentral'

### Task 2
Site Deployment script site.py

A typical Day-1 workflow is to deploy a new site. This site will have devices with some goldend configuration to get the network up and running. When it comes to Aruba Central there are certain steps that network administrators would like to perfor as part of the Day-1 Automation as below:

- Create a new group - to ensure that common configuration for specific devices is stored at the group level
- Add configuration tempaltes - to upload valid configuration template that devices will use
- Add device variables - to define reusable variable values used in a configuration template
- Create a new site - to combine devices with respect to their geographical location for monitoring purposes
- Add devices to the newly created group (optional step)
- Associating devices to the newly created site (optional step)

Note: Above mentioned optional steps can only be performed if you a device online on central as well as its device specific (AP, Switch or Gateway) configuration template.

In [ ]:
# We will use the base module from pycentral package in this script alongwith Configuration and Monitoring module
from pycentral.base import ArubaCentralBase
# Import Groups , Devices, Templates and Sites class
from pycentral.configuration import Groups, Devices, Templates
from pycentral.monitoring import Sites


# Create an instance of ArubaCentralBase using API Gateway's Base URL and the API access token

central_info = {
    "base_url": "<api-gateway-base-url>",
    "token": {
        "access_token": "<api-gateway-access-token>"
    }
}
ssl_verify = True
central = ArubaCentralBase(central_info=central_info,
                           ssl_verify=ssl_verify)

# Instantiating the classes
g = Groups()
s = Sites()
d = Devices()
t = Templates()

# Define Workflow Variables
group_name = input("Enter Group Name: ")
group_pass = input("Enter Group Password: ")
template_name = input("Enter Template Name: ")
template_filename = input("Enter absolute file path where configuration "
                          "template is stored on your system: ")
# absolute file path example: "/home/admin/automation_folder/central/templates/iap-temp.txt"
site_name = input("Enter Site Name")
site_address = {
  "address": input("Enter Street Address: "),
  "city": input("Enter City: "),
  "state": input("Enter State (fullname): "),
  "country": input("Enter Country (country code): "),
  "zipcode": input("Enter Zipcode: ")
}
device_type = input("Enter Device Type (IAP/CX/MobilityController): ")
device_serials = []
device_serials.append(input("Enter Device Serial Number(for one device): "))


# 1. Create a new Template Group
print("1. Creating Group...")
resp = g.create_group(central, group_name=group_name, group_password=group_pass, 
                      wired_template=True, wireless_template=True)
print(resp)

# 2. Upload a new Configuration Template
print("2. Uploading Template...")
resp = t.create_template(central, group_name, template_name, template_filename,
                         device_type, version="ALL", model="ALL")
print(resp)

# 3. Move devices to a Group
print("4. Moving device to new Group")
resp = d.move_devices(central, group_name=group_name,
                      device_serials=device_serials)
print(resp)  
    
# 4. Create a Site
print("2. Creating Site")
resp = s.create_site(central, site_name=site_name, site_address=site_address)
print(resp)

Once the above script is run, you should see an output as below:


1. Creating Group...
{'code': 201, 'msg': 'Created', 'headers': {'Content-Type': 'application/json', 'Content-Length': '10', 'Connection': 'keep-alive', 'X-RateLimit-Limit-day': '1000', 'X-RateLimit-Remaining-day': '959', 'X-RateLimit-Limit-second': '7', 'X-RateLimit-Remaining-second': '6', 'Date': 'Tue, 19 Oct 2021 07:18:24 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, no-store, must-revalidate, private', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'X-Request-Start': 't=1635837504.703', 'X-XSS-Protection': '1; mode=block', 'X-Kong-Upstream-Latency': '886', 'X-Kong-Proxy-Latency': '5', 'Via': 'kong/0.14.1'}}

2. Uploading Template...
{'code': 201, 'msg': 'Created', 'headers': {'Content-Type': 'application/json', 'Content-Length': '10', 'Connection': 'keep-alive', 'X-RateLimit-Limit-day': '1000', 'X-RateLimit-Remaining-day': '958', 'X-RateLimit-Limit-second': '7', 'X-RateLimit-Remaining-second': '6', 'Date': 'Tue, 19 Oct 2021 07:18:25 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, no-store, must-revalidate, private', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'X-Request-Start': 't=1635837505.229', 'X-XSS-Protection': '1; mode=block', 'X-Kong-Upstream-Latency': '401', 'X-Kong-Proxy-Latency': '2', 'Via': 'kong/0.14.1'}}

3. Adding device to new Group
{'code': 200, 'msg': 'Success', 'headers': {'Content-Type': 'application/json', 'Content-Length': '10', 'Connection': 'keep-alive', 'X-RateLimit-Limit-day': '1000', 'X-RateLimit-Remaining-day': '957', 'X-RateLimit-Limit-second': '7', 'X-RateLimit-Remaining-second': '6', 'Date': 'Tue, 19 Oct 2021 07:18:26 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, no-store, must-revalidate, private', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'X-Request-Start': 't=1635837506.066', 'X-XSS-Protection': '1; mode=block', 'X-Kong-Upstream-Latency': '700', 'X-Kong-Proxy-Latency': '1', 'Via': 'kong/0.14.1'}}

4. Creating Site
{'code': 200, 'msg': {'address': '560 Oakmead Parkway', 'city': 'Sunnyvale', 'country': 'United States', 'latitude': '37.3860849', 'longitude': '-121.9877437', 'site_id': 157, 'site_name': 'Site-A', 'state': 'California', 'zipcode': '94085'}, 'headers': {'Content-Type': 'application/json', 'Content-Length': '241', 'Connection': 'keep-alive', 'X-RateLimit-Limit-day': '1000', 'X-RateLimit-Remaining-day': '956', 'X-RateLimit-Limit-second': '7', 'X-RateLimit-Remaining-second': '6', 'Date': 'Tue, 19 Oct 2021 07:18:28 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, no-store, must-revalidate, private', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'X-Request-Start': 't=1635837508.018', 'X-XSS-Protection': '1; mode=block', 'X-Kong-Upstream-Latency': '1820', 'X-Kong-Proxy-Latency': '2', 'Via': 'kong/0.14.1'}}

5. Associating device to new Site
{'code': 200, 'msg': '', 'headers': {'Content-Type': 'application/json', 'Content-Length': '0', 'Connection': 'keep-alive', 'X-RateLimit-Limit-day': '1000', 'X-RateLimit-Remaining-day': '955', 'X-RateLimit-Limit-second': '7', 'X-RateLimit-Remaining-second': '6', 'Date': 'Tue, 19 Oct 2021 07:18:29 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, no-store, must-revalidate, private', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'X-Request-Start': 't=1635837509.886', 'X-XSS-Protection': '1; mode=block', 'X-Kong-Upstream-Latency': '1733', 'X-Kong-Proxy-Latency': '5', 'Via': 'kong/0.14.1'}}


As you can see for each of the steps we exeucted in the script we are receving a 2XX series response, signifying that all steps were executed without any issues.

### What did we learn today?

- Different API available on Central
- Central's API Gateway
- API documentation
- PyCentral Python SDK
- How build automated workflows and make API calls using PyCentral